# utilities

> Utilitiy functions for dealing with WEAVE data

In [ ]:
# |default_exp utilities

In [ ]:
# |export

import numpy as np
import xarray as xr
from astropy.time import Time
from scipy import stats

## Parsing codes in the FITS headers

The header entry "OBSTEMP" encodes the requested observational constraints for the OB: the seeing, transparency, elevation, lunar angle and sky brightness. The OBSTEMP code consists of a letter for each constraint. This converts the codes to numerical values and adds airmass.

In [ ]:
# |export


def parse_obstemp(
    codes: xr.DataArray,  # a DataArray of OBSTEMP codes
) -> dict:  # the numerical value for each observing constraint
    """Convert an OBSTEMP `code` to a dictionary of constraint values."""
    lookup = dict(
        seeing={
            "A": 0.7,
            "B": 0.8,
            "C": 0.9,
            "D": 1.0,
            "E": 1.1,
            "F": 1.2,
            "G": 1.3,
            "H": 1.4,
            "I": 1.5,
            "J": 1.6,
            "K": 1.7,
            "L": 1.8,
            "M": 1.9,
            "N": 2.0,
            "O": 2.1,
            "P": 2.2,
            "Q": 2.3,
            "R": 2.4,
            "S": 2.5,
            "T": 2.6,
            "U": 2.7,
            "V": 2.8,
            "W": 2.9,
            "X": 3.0,
        },
        transparency={"A": 0.8, "B": 0.7, "C": 0.6, "D": 0.5, "E": 0.4},
        elevation={
            "A": 50.28,
            "B": 45.58,
            "C": 41.81,
            "D": 35.68,
            "E": 33.75,
            "F": 25.00,
        },
        lunar_angle={"A": 90, "B": 70, "C": 50, "D": 30, "E": 0},
        sky_brightness={
            "A": 21.7,
            "B": 21.5,
            "C": 21.0,
            "D": 20.5,
            "E": 19.6,
            "F": 18.5,
            "G": 17.7,
        },
    )

    cols = dict()
    for i, k in enumerate(lookup):
        v = [lookup[k][c[i]] for c in codes.to_numpy()]
        cols[k] = xr.Variable(codes.dims, v)
    cols["airmass"] = np.round(1 / np.cos(np.radians(90 - cols["elevation"])), 1)
    return xr.Dataset(cols, codes.coords)

## Parsing line names

Line names in the galaxy table are a combination of the species and restframe wavelength, e.g. `[OIII]_5006.77`. This splits them and returns the species and wavelength separately.

In [ ]:
# |export


def parse_line_names(
    lines: xr.DataArray,  # a DataArray of line names, e.g. [OIII]_5006.77
) -> tuple[xr.DataArray, xr.DataArray]:  # the species and wavelength of each line
    """Split the line names into species and restframe wavelength."""
    lines = lines.astype(str)
    line_species, line_rest_wl = lines.str.split(dim="", sep="_").T
    line_rest_wl = line_rest_wl.astype(float)
    return line_species, line_rest_wl

## Statistics

In [ ]:
# |export


def ks_norm_prob(
    x: xr.DataArray,  # a 2D array of values to test
    dim="SPEC",  # the dimension of `x` to retain
    cdf=stats.norm.cdf,  # the cdf to compare against
) -> xr.DataArray:
    """Calculate the KS probability that `x` are drawn from a standard Normal distribution.

    The implementation is based on `scipy.stats.ks_1samp` and adapted to xarray for speed.

    The input DataArray must be 2D, with one dimension to be kept (specified by `dim`) and
    the other for which the distribution of values will be tested. A multi-dimensional
    DataArray may be converted to 2D using `DataArray.stack`.
    """
    exdim = [d for d in x.dims if d != dim][0]
    n_pix, n_data = x.shape
    n_valid = x.count(dim=exdim)
    x = xr.apply_ufunc(
        np.sort,
        x.reset_index(exdim, drop=True),
        input_core_dims=[[exdim]],
        output_core_dims=[[exdim]],
        dask="parallelized",
    )
    cdfvals = xr.apply_ufunc(cdf, x, dask="parallelized")
    with np.errstate(divide="ignore", invalid="ignore"):
        dplus = np.arange(1, n_pix + 1) / n_valid.data[:, None] - cdfvals
        dminus = cdfvals - np.arange(0, n_pix) / n_valid.data[:, None]
    Dplus = dplus.max(axis=-1)
    Dminus = dminus.max(axis=-1)
    D = np.maximum(Dplus, Dminus)
    with np.errstate(divide="ignore", invalid="ignore"):
        prob = xr.apply_ufunc(stats.kstwo.sf, D, n_valid, dask="parallelized")
    prob = prob.clip(0, 1)
    return prob

## Time

In [ ]:
# |export


def mjd_to_night(
    mjd: float | np.ndarray,  # a Modified Julian Date
) -> float | np.ndarray:  # the night in YYYYMMDD format
    """Return the date (YYYYMMDD) at the start of the night containing `mjd`."""
    t = Time(mjd - 0.5, format="mjd")
    return t.to_value("iso", subfmt="date").replace("-", "")

In [ ]:
assert mjd_to_night(57639.8653) == "20160908"

In [ ]:
assert mjd_to_night(57639.8653) == mjd_to_night(57640.1234)

In [ ]:
# |hide
import nbdev

nbdev.nbdev_export()